#### Setup


In [6]:
import torch
import torchvision
from lib.data_handlers import Load_ImageNet100, Load_PACS, Load_ImageNet100Sketch
from overcomplete.models import DinoV2, ViT, ResNet, ViT_Large, SigLIP
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae
from overcomplete.visualization import (overlay_top_heatmaps, evidence_top_images, zoom_top_images, contour_top_image)
import os
import matplotlib.pyplot as plt
from einops import rearrange
from lib.universal_trainer import train_usae
from lib.activation_generator import Load_activation_dataloader
import torch.nn as nn
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
from lib.eval import evaluate_models
from lib.visualizer import visualize_concepts
from tqdm import tqdm
import timm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


## ImageNet

#### Train SAE

In [7]:
models = {
    "ViT": ViT(device="cuda")
}

activations_dir="/activations/ViT_Solo"

image_loader = Load_ImageNet100(transform=None, batch_size=256, shuffle=True)

activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=image_loader,
    max_seq_len=196,
    save_dir=activations_dir,
    generate=False,
    rearrange_string='n t d -> (n t) d'
)

Set Parameters

In [8]:
concepts = 768 * 8
epochs = 100
lr=3e-4
sample = next(iter(activations_dataloader))

Execute Flow

In [9]:
SAEs = {}
optimizers = {}
schedulers = {}

for key, model in models.items():

    SAEs[key] = TopKSAE(
        sample[f"activations_{key}"].shape[-1],
        nb_concepts=concepts,
        top_k=16,
        device="cuda",
    )
    optimizers[key] = torch.optim.Adam(SAEs[key].parameters(), lr=lr)

    # Set up a Linear + Cosine Scheduler
    warmup_scheduler = LinearLR(
        optimizers[key], start_factor=1e-6 / 3e-4, end_factor=1.0, total_iters=10
    )
    cosine_scheduler = CosineAnnealingLR(optimizers[key], T_max=epochs, eta_min=1e-6)
    schedulers[key] = SequentialLR(
        optimizers[key],
        schedulers=[warmup_scheduler, cosine_scheduler],
        milestones=[25],
    )

criterion = nn.L1Loss(reduction="mean")  # change to mean reduction

In [6]:
train_usae(
    names=list(models.keys()),
    models=SAEs,
    dataloader=activations_dataloader,
    criterion=criterion,
    nb_epochs=epochs,
    optimizers=optimizers,
    schedulers=schedulers,
    device="cuda",
    seed=42,
)

Epoch 1/100:   5%|▍         | 24/508 [00:07<02:22,  3.40it/s, loss=0.718]c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\torch\optim\lr_scheduler.py:198: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
Epoch 1/100: 100%|██████████| 508/508 [02:28<00:00,  3.41it/s, loss=0.47] 



[Epoch 1] Loss: 275.5159 | Time: 148.84s | Dead Features: 0.0%


Epoch 2/100: 100%|██████████| 508/508 [02:28<00:00,  3.42it/s, loss=0.43] 



[Epoch 2] Loss: 230.0995 | Time: 148.54s | Dead Features: 0.1%


Epoch 3/100: 100%|██████████| 508/508 [02:29<00:00,  3.40it/s, loss=0.436]


...


[Epoch 98] Loss: 188.2775 | Time: 152.21s | Dead Features: 72.0%


Epoch 99/100: 100%|██████████| 508/508 [02:31<00:00,  3.35it/s, loss=0.377]



[Epoch 99] Loss: 188.3318 | Time: 151.61s | Dead Features: 71.7%


Epoch 100/100: 100%|██████████| 508/508 [02:32<00:00,  3.32it/s, loss=0.367]


[Epoch 100] Loss: 188.2568 | Time: 152.88s | Dead Features: 71.7%


In [ ]:
model_state_dicts = {name: model.state_dict() for name, model in SAEs.items()}
torch.save(model_state_dicts, "./models/ViT_MLP.pt")

: 

In [10]:
model_path = "./models/ViT_MLP.pt"
state = torch.load(model_path)

for name, sae in SAEs.items():
    print(sae.load_state_dict(state[name]))

<All keys matched successfully>


In [ ]:
### SAVE THE Z_sAES and then on them
visualize_concepts(
    activation_loader=activations_dataloader,
    SAEs=SAEs,
    num_concepts=concepts,
    n_images=8,
    patch_width=14, 
    save_dir="results/visualizer_ViT_MLP/",
    abort_threshold=0.0,
)

100%|██████████| 508/508 [14:12<00:00,  1.68s/it]


6144
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipping cuz 0
Skipp

In [ ]:
vit_full = timm.create_model('vit_base_patch16_224', pretrained=True)
mlp = nn.Linear(concepts, 1000) # 1000 Classes
vit_extractor = models["ViT"]

##################



##################


